# Perform OAE experiment using GLORYS12V1 BGC reconstruction
Created by Ivan Lima on Thu Jul 28 2022 13:42:31 -0400

In [1]:
%matplotlib inline
import xarray as xr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, datetime
from tqdm import notebook
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Thu Jul 28 16:02:08 2022


In [2]:
import PyCO2SYS as pyco2
import gsw

datadir = '/bali/data/ilima/GLORYS12V1/daily/BGC/surface'
file = 'GLORYS12V1_NW_Atlantic_2019_surf_pCO2.nc'
year = file.split('_')[3]
ds = xr.open_dataset(os.path.join(datadir, file))

depth = 0.494024991989136 # depth of surface layer
delta_TA = 500.

df = ds[['SST','SSS','DIC','TA']].to_dataframe().reset_index()
df['TA'] = df.TA + delta_TA
df_data = df.copy().dropna()
df_data.head()

,time,latitude,longitude,SST,SSS,DIC,TA
2,2019-01-01 12:00:00,34.0,-77.833336,13.734886,33.144627,2013.113403,2729.367676
3,2019-01-01 12:00:00,34.0,-77.750000,13.819849,33.539841,2017.723511,2747.386230
4,2019-01-01 12:00:00,34.0,-77.666664,14.072542,34.072392,2038.391235,2774.213379
5,2019-01-01 12:00:00,34.0,-77.583336,14.410199,34.575947,2055.984863,2798.602539
6,2019-01-01 12:00:00,34.0,-77.500000,14.772026,34.900967,2066.742432,2814.122070


In [3]:
pressure =  gsw.p_from_z(-depth, df_data.latitude.values)
kwargs = dict(
    par1 = df_data.TA.values ,  # TA
    par2 = df_data.DIC.values,  # DIC
    par1_type = 1,              # type 1 = alkalinity
    par2_type = 2,              # type 2 = DIC
    salinity = df_data.SSS.values,
    temperature = df_data.SST.values,
    pressure = pressure,
    opt_k_carbonic = 10,  # LDK00, Lueker et al 2000
    opt_k_bisulfate = 1,  # D90a, Dickson 1990
    opt_total_borate = 2, # LKB10, Lee et al 2010
    opt_k_fluoride = 2    # PF87, Perez & Fraga 1987
)
results = pyco2.sys(**kwargs)
co2sys_vars = ['pCO2', 'fCO2']
for vname in co2sys_vars:
    df_data[vname] = results[vname]

# merge estimated carbonate chemistry variables to original dataframe
for vname in co2sys_vars:
    df[vname] = df_data[vname]

df.head()

,time,latitude,longitude,SST,SSS,DIC,TA,pCO2,fCO2
0,2019-01-01 12:00:00,34.0,-78.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 12:00:00,34.0,-77.916664,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 12:00:00,34.0,-77.833336,13.734886,33.144627,2013.113403,2729.367676,68.886316,68.633531
3,2019-01-01 12:00:00,34.0,-77.750000,13.819849,33.539841,2017.723511,2747.386230,68.077637,67.828091
4,2019-01-01 12:00:00,34.0,-77.666664,14.072542,34.072392,2038.391235,2774.213379,70.600014,70.342052


In [6]:
ds_out = df.set_index(['time','latitude','longitude']).to_xarray()
# copy variable attributes
for vname in ['SST','SSS','DIC','TA','pCO2','fCO2']:
    ds_out[vname].attrs.update(ds[vname].attrs)

outdir = '/bali/data/ilima/GLORYS12V1/daily/BGC/surface/TA_500'
outfile = os.path.join(outdir, 'GLORYS12V1_NW_Atlantic_{}_surf_OAE_TA500.nc'.format(year))
print('writing {}'.format(outfile))
ds_out.to_netcdf(outfile, mode='w', unlimited_dims=['time'],
                 encoding={'TA': {'dtype':'float32'}, 'pCO2': {'dtype':'float32'}, 'fCO2': {'dtype':'float32'}})

writing /bali/data/ilima/GLORYS12V1/daily/BGC/surface/TA_500/GLORYS12V1_NW_Atlantic_2019_surf_OAE_TA500.nc
